In [2]:
from io import BytesIO
from urllib import request

import numpy as np
import onnx
import onnxruntime as rt
from PIL import Image

In [ ]:
sess = rt.InferenceSession(
    "./data/hair_classifier_v1.onnx", providers=rt.get_available_providers()
)
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name

# Question 1

In [18]:
print(f"The name of the output is '{output_name}'")

The name of the output is 'output'


# Question 2

In [ ]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != "RGB":
        img = img.convert("RGB")
    img = img.resize(target_size, Image.NEAREST)
    return img

We need to resize the image to 200x200

In [ ]:
url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
target_size = (200, 200)
raw_image = download_image(url)
image = prepare_image(raw_image, target_size)

# Question 3

In [22]:
image_arr = np.asarray(image)

In [ ]:
def normalize_data(
    image_arr: np.ndarray, means: np.ndarray, stds: np.ndarray
) -> np.ndarray:
    # We must first convert from [0,255] to [0,1]  - transforms.ToTensor(),
    # Now we normalize the data to zero mean and unit variance - transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    image_arr = image_arr / 255
    return (image_arr - means) / stds


means = np.array([0.485, 0.456, 0.406])
stds = np.array([0.229, 0.224, 0.225])

In [ ]:
image_n = normalize_data(image_arr, means, stds).astype(np.float32)

In [64]:
print(f"The value of the first pixel in the R channel is {image_n[0,0,0]}")

The value of the first pixel in the R channel is -1.073293924331665


# Question 4

In [66]:
# Reshape the input
# Original: (200,200,3) in W,H,C
# Then (1,200,200,3) in B,W,H,C
# Final (1,3,200,200) in B,C,W,H
image_input = image_n.reshape(1, 200, 200, 3).transpose(0, 3, 1, 2)

In [63]:
sess.run(["output"], {"input": image_input})

[array([[0.09156641]], dtype=float32)]